In [1]:
import numpy as np
from scipy.stats import rv_continuous
class gaussian_gen(rv_continuous):
    def _pdf(self, x):
        return np.exp(-x**2 / 2.) / np.sqrt(2.0 * np.pi)
gaussian = gaussian_gen(name='gaussian')
gaussian.mean()

-5.955875168377274e-16

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ddm
import os 
import csv
import helper
from model_definitions import ModelTtaBounds

In [2]:
for folder in ['cross_validation_all', 'cross_validation_8', 'cross_validation_4']:
    helper.merge_csv('../model_fit_results/%s' % (folder))

../model_fit_results/cross_validation_all\subj_129.csv
../model_fit_results/cross_validation_all\subj_389.csv
../model_fit_results/cross_validation_all\subj_525.csv
../model_fit_results/cross_validation_all\subj_616.csv
../model_fit_results/cross_validation_all\subj_618.csv
../model_fit_results/cross_validation_all\subj_642.csv
../model_fit_results/cross_validation_all\subj_755.csv
../model_fit_results/cross_validation_all\subj_all.csv
../model_fit_results/cross_validation_8\subj_129.csv
../model_fit_results/cross_validation_8\subj_389.csv
../model_fit_results/cross_validation_8\subj_525.csv
../model_fit_results/cross_validation_8\subj_616.csv
../model_fit_results/cross_validation_8\subj_618.csv
../model_fit_results/cross_validation_8\subj_642.csv
../model_fit_results/cross_validation_8\subj_755.csv
../model_fit_results/cross_validation_8\subj_all.csv
../model_fit_results/cross_validation_4\subj_129.csv
../model_fit_results/cross_validation_4\subj_389.csv
../model_fit_results/cross_val

In [15]:
def get_model_measures(model, condition):
    sol = model.solve(condition)
    return(condition['tta_condition'], condition['d_condition'], sol.prob_correct(), sol.mean_decision_time())

def get_model_rt_cdf(model, condition):
    sol = model.solve(condition)
    return(pd.DataFrame({'tta_condition': condition['tta_condition'],
                         'd_condition':  condition['d_condition'],
                         't': model.t_domain(),
                         'rt_corr_pdf': sol.cdf_corr()/sol.prob_correct()}))

def get_model_rt_pdf(model, condition):
    sol = model.solve(condition)
    return(pd.DataFrame({'tta_condition': condition['tta_condition'],
                         'd_condition':  condition['d_condition'],
                         't': model.t_domain(),
                         'rt_corr_pdf': sol.pdf_corr()/sol.prob_correct()}))
#     return(condition['tta_condition'], condition['d_condition'], sol.pdf_corr()/sol.prob_correct())

def initialize_model(param_set):
    overlay = ModelTtaBounds.OverlayNonDecisionGaussian(nondectime=param_set.ndt_location, ndsigma=param_set.ndt_scale)
    model = ddm.Model(name='TTA- and d-dependent drift and bounds and random nondecision time',
                         drift=ModelTtaBounds.DriftTtaDistance(alpha=param_set.alpha, 
                                                               beta=param_set.beta, theta=param_set.theta),
                         noise=ddm.NoiseConstant(noise=param_set.noise),
                         bound=ModelTtaBounds.BoundCollapsingTta(b_0=param_set.b_0, 
                                                                 k=param_set.k, tta_crit=param_set.tta_crit),
                         overlay=overlay, T_dur=ModelTtaBounds.T_dur)
    return(model)

def simulate_model(param_set, conditions, ret='measures'): 
    print(param_set.subj_id)
    model = initialize_model(param_set)
    if ret=='measures':
        model_measures = pd.DataFrame([get_model_measures(model, condition) for condition in conditions],
                                  columns=['tta_condition', 'd_condition', 'is_turn_decision', 'RT'])
        model_measures['subj_id'] = param_set.subj_id
        return(model_measures)
    elif ret=='rt_pdf':
        model_rt_pdf = pd.concat([get_model_rt_pdf(model, condition) for condition in conditions])
        model_rt_pdf['subj_id'] = param_set.subj_id
        return(model_rt_pdf)
    elif ret=='rt_cdf':
        model_rt_cdf = pd.concat([get_model_rt_cdf(model, condition) for condition in conditions])
        model_rt_cdf['subj_id'] = param_set.subj_id
        return(model_rt_cdf)
    else:
        raise(ValueError('ret value should be either "measures" or "rt_pdf"'))

In [3]:
def save_sim_results(file_name, conditions=None, ret='measures'):
    path = '../model_fit_results/'
    file_path = os.path.join(path, file_name)
    parameters = pd.read_csv(file_path).rename(columns={'tta': 'tta_condition', 'd': 'd_condition'})
    
    if ret=='measures':
        if 'all' in file_name:
            model_measures = [simulate_model(param_set, conditions, ret='measures') 
                                  for idx, param_set in parameters.iterrows()]
        else:
            model_measures = [simulate_model(param_set, [param_set[['tta_condition', 'd_condition']].to_dict()]) 
                               for idx, param_set in parameters.iterrows()]
        model_measures = pd.concat(model_measures)
        model_measures.to_csv(file_path.replace('parameters_fitted', 'measures'), index=False)
    else:
        model_rt_pdf = [simulate_model(param_set, conditions, ret=ret) for idx, param_set in parameters.iterrows()]
        model_rt_pdf = pd.concat(model_rt_pdf)
        model_rt_pdf.to_csv(file_path.replace('parameters_fitted', ret), index=False)

In [6]:
conditions = [{'tta_condition': tta, 'd_condition': d} 
                       for tta in [4,5,6]
                       for d in [90,120,150]]

In [ ]:
save_sim_results('cross_validation_all_parameters_fitted.csv', conditions, ret='rt_cdf')

In [ ]:
save_sim_results('cross_validation_all_parameters_fitted.csv', conditions, ret='rt_pdf')

In [7]:
save_sim_results('cross_validation_all_parameters_fitted.csv', conditions, ret='measures')

## Simulating for cross-validation

In [ ]:
save_sim_results('cross_validation_8_parameters_fitted.csv', conditions, ret='measures')